In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Files
import input_net
import utils

# Packages
from os import path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# Transformers
from transformers import AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer
from transformers import BartConfig, BartForSequenceClassification, BartTokenizer
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
from transformers import CamembertConfig, CamembertForSequenceClassification, CamembertTokenizer
from transformers import DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import FlaubertConfig, FlaubertForSequenceClassification, FlaubertTokenizer
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer
from transformers import XLMConfig, XLMForSequenceClassification, XLMTokenizer
from transformers import XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from transformers import XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer
from sklearn.model_selection import train_test_split

from transformers import AdamW, get_linear_schedule_with_warmup

Using TensorFlow backend.


In [3]:
args = {
    'model_type': 'bert',
    'do_train': True,
    'do_eval': True,
    'max_seq_length': 70,
    'batch_size': 5, 
    'epochs': 1,
    'learning_rate': 4e-5,
    'num_training_steps': 1000,
    'num_warmup_steps': 100,
    'max_grad_norm': 1.0
}

In [10]:
MODEL_CLASSES = {
    'albert': (AlbertForSequenceClassification, AlbertTokenizer, 'albert-large-v2'),
    'bart': (BartForSequenceClassification, BartTokenizer, 'bart-large'),
    'bert': (BertForSequenceClassification, BertTokenizer, 'bert-base-uncased'),
    'camembert': (CamembertForSequenceClassification, CamembertTokenizer, 'camembert-base'),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, 'distilbert-base-uncased'),
    'flaubert': (FlaubertForSequenceClassification, FlaubertTokenizer, 'flaubert-base-uncased'),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, 'roberta-base'),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, 'xlm-mlm-en-2048'),
    'xlm roberta': (XLMRobertaForSequenceClassification, XLMRobertaTokenizer, 'xlm-roberta-base'),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, 'xlnet-base-cased')
}

model_class, tokenizer_class, pretrained_model = MODEL_CLASSES[args['model_type']]

In [11]:
TRAIN = "data/train.csv"
TEST = "data/test.csv"
INPUT_NET = 'data/input.csv'

tokenizer = tokenizer_class.from_pretrained(pretrained_model, do_lower_case=True)

if not path.exists(INPUT_NET):
    df = input_net.create_input(TRAIN, INPUT_NET, tokenizer, args['max_seq_length'])
else:
    df = pd.read_csv(INPUT_NET)

In [18]:
def batch_generator(data, target, batch_size):
    data = np.array(data)
    target = np.array(target)
    nsamples = len(data)
    perm = np.random.permutation(nsamples)
    for i in range(0, nsamples, batch_size):
        batch_idx = perm[i:i+batch_size]
        if target is not None:
            #print("data:", data.loc[batch_idx,:])
            yield data[batch_idx,:], target[batch_idx]
        #else:
        #    yield data[batch_idx], None

def training(model, train_data, train_target, batch_size, args):
    criterion = nn.CrossEntropyLoss(reduction='mean')
    optimizer = AdamW(model.parameters(), lr=args['learning_rate'], correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['num_warmup_steps'],
                                                num_training_steps=args['num_training_steps'])
    ncorrect = 0
    total_loss = 0
    for X, y in batch_generator(train_data, train_target, batch_size):
        model.train()
        X_i, X_s, X_p, y = utils.ToTensor(X,y)
        
        #model.zero_grad()
        out = model(input_ids=X_i, token_type_ids=X_s, attention_mask=X_p, labels=y)[1]
        
        loss = criterion(out, y)
        loss.backward()
        total_loss += loss
        #torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])  # Gradient clipping 
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        out = F.softmax(out, dim=1)

        ncorrect += (torch.max(out, 1)[1] == y).sum().item()
        print("train:", loss.item())
    total_loss /= len(train_data)
    acc = ncorrect/len(train_data) * 100
    return acc, loss
        
#def validation(model, eval_data, batch_size):


    

In [15]:
def build(model, learn_data, batch_size, epochs, args):
    X_train, X_val, y_train, y_val = train_test_split(learn_data.iloc[:,:-1], learn_data.iloc[:,-1],                                                            test_size=0.2, random_state=42)
    train_acc = [None]*epochs
    train_loss = [None]*epochs
    val_acc = [None]*epochs
    val_los = [None]*epochs
    for epoch in range(epochs):
        acc, loss = training(model, X_train, y_train, batch_size, args)
             
        train_acc[epoch] = acc
        train_loss[epoch] = loss
        
        #acc, loss = validation(model, val_data)
        val_acc[epoch] = acc
        val_loss[epoch] = loss
        print(acc,loss)

In [16]:
model = model_class.from_pretrained(pretrained_model, num_labels=2)
print("model loaded")
build(model, df, batch_size = args['batch_size'], epochs = args['epochs'], args=args)

model loaded
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.9740232229232788
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.9974401593208313
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.8854871988296509
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.9193907976150513
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.7468396425247192
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.8513696789741516
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.7590999007225037
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.6302300691604614
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.6549879312515259
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.7711094617843628
torch.Size([5, 70]) torch.Size([5, 70]) torch.Size([5, 70])
train: 0.6099561452865601
torch.Size([5, 70]) torch.Size([5, 70]) t

KeyboardInterrupt: 

In [13]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [9]:
#29361, 28995

In [11]:
print(len(df))

404290


In [15]:
Predictions = model(input_ids=tokens_tensor[:5],token_type_ids=segments_tensor[:5],attention_mask=attention_tensor[:5])

NameError: name 'tokens_tensor' is not defined

In [31]:
F.softmax(model(input_ids=tokens_tensor[:5],token_type_ids=segments_tensor[:5],attention_mask=attention_tensor[:5])[0], dim=1)

tensor([[0.7353, 0.2647],
        [0.7387, 0.2613],
        [0.7226, 0.2774],
        [0.6201, 0.3799],
        [0.7045, 0.2955]], grad_fn=<SoftmaxBackward>)

In [59]:
Predictions[1]

tensor([[-0.3982,  0.6718],
        [-0.4118,  0.6909],
        [-0.3936,  0.6663],
        [-0.4014,  0.6766],
        [-0.4046,  0.6803]], grad_fn=<AddmmBackward>)